In [3]:
import os
import cv2
import albumentations as A

# Define the augmentation pipeline
transform = A.Compose([
    A.Rotate(limit=30, p=0.5),  # 1. Rotation
    A.RandomScale(scale_limit=0.2, p=0.5),  # 2. Scaling
    A.HorizontalFlip(p=0.5),  # 4. Horizontal Flip
    A.GaussNoise(var_limit=(10, 100), p=0.5),  # 5. Gaussian Noise
])

def apply_augmentation(input_folder, output_folder, num_augs_per_image=2):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        for i in range(num_augs_per_image):
            augmented_image = transform(image=image)['image']
            augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(os.path.join(output_folder, f"{os.path.splitext(img_name)[0]}_aug_{i}.jpg"), 
                        augmented_image, [cv2.IMWRITE_JPEG_QUALITY, 85])

def augment_dataset(base_input_path, base_output_path, num_augs_per_image=2):
    # Iterate through train, test, and val folders
    for dataset_type in ['train', 'test', 'validation']:
        input_path = os.path.join(base_input_path, dataset_type)
        output_path = os.path.join(base_output_path, dataset_type)

        # Iterate through normal and pneumonia subfolders
        for category in os.listdir(input_path):
            cat_input_path = os.path.join(input_path, category)
            cat_output_path = os.path.join(output_path, category)
            
            apply_augmentation(cat_input_path, cat_output_path, num_augs_per_image)

# Example usage:
# Note: Adjust the paths according to your project structure.
augment_dataset("data_to_model", "data_to_model_augmented")
